Importamos las dependencias necesarias.

In [1]:
# Para cargar datasets desde Google Drive
from google.colab import drive

# Para preprocesar los datasets
import numpy as np
from numpy import asarray, zeros, random
import pandas as pd

# Para dividir el conjunto de datos en train y test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample, shuffle

# Para aprender word embeddings
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

# Para construir el modelo
from keras.models import Sequential,Model
from keras.layers import Dense, Embedding, LSTM, GRU, Flatten, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras.utils import np_utils
from keras import optimizers

# Para representaciones gráficas
import matplotlib.pyplot as plt

# Otras operaciones
import re

Using TensorFlow backend.


Cargamos desde Google Drive el dataset preprocesado con OpenRefine.

In [2]:
drive.mount('/content/gdrive')

data = pd.read_csv('/content/gdrive/My Drive/Hotel_BINARY.csv')
data.head(3)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


,Review,Reviewer_Score
0,I am so angry that i made this post available...,0
1,No Negative No real complaints the hotel was g...,1
2,Rooms are nice but for elderly a bit difficul...,1


In [0]:
seed = 14

Dado que sólo vamos a usar una pequeña parte de registros para el entrenamiento (entre 8000 y 15000) por modelo, dividimos el dataset en training y test. Este último tendrá 3000 registros para agilizar el proceso de testing posteriormente.

In [0]:
# División en training y test
train, test = train_test_split(data, test_size=0.02, random_state = seed)

# Usaremos sólo los 3000 primeros registros de la parte de test
test = test.iloc[:3000]

# Longitud máxima para las reviews
max_length = 600

Un elemento fundamental es el Tokenizer, que nos permite convertir cada palabra en un número o conjunto de ellos, para que puedan servir de entrada a la red neuronal que se creará más adelante.

In [0]:
#
max_features = 1000
#
tokenizer = Tokenizer(num_words=max_features, split=' ')
#
tokenizer.fit_on_texts(data['Review'].values)
#
vocab = len(tokenizer.word_index)

En nuestro ensemble, dos de los modelos estarán creados con la ayuda de los vectores de palabras de GloVe, que han sido preentrenados y se encuentran disponibles para su descarga desde su [página oficial](https://nlp.stanford.edu/projects/glove/). Por motivos de recursos, usaremos su versión más pequeña, disponible para su descarga [aquí](http://nlp.stanford.edu/data/glove.6B.zip). Esta versión ha sido entrenada a partir de textos extraidos de Wikipedia y de varias plataformas de noticias internacionales, como el New York Times o el Washington Post.

In [11]:
embeddings_index = dict()
f = open('/content/gdrive/My Drive/glove.6B.50d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab, 50))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


Ya que vamos a crear varios modelos de redes neuronales, es necesario guardarlas en una lista, así como sus hiperparámetros.

In [0]:
models = []
n_registers = []
batch_sizes = []

La primera red neuronal tiene una estructura simple, formada unicamente por una capa Dropout y una capa LSTM. Como aclaración, la capa de Dropout es un elemento que ayuda a mitigar el sobreajuste, desactivando aleatoriamente un porcentaje definido de enlaces durante cada paso de entrenamiento. Definimos su estructura y la compilamos.

In [13]:
# Hiperparámetros de la red E0
embed_dim = 128
lstm_out = 196

# Inicialización
model0 = Sequential()
# Capa de entrada
model0.add(Embedding(max_features, embed_dim,input_length = max_length))
# Capa Dropout
model0.add(SpatialDropout1D(0.4))
# Capa LSTM
model0.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# Capa con dos salidas
#   Salida 0: emociones negativas
#   Salida 1: emociones positivas
model0.add(Dense(2,activation='softmax'))
# Compilamos el modelo
model0.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# Mostramos su información
print(model0.summary())

# Guardamos el modelo en la lista de modelos, al igual que el tamaño de batch y el número de registros
models.append(model0)
batch_sizes.append(128)
n_registers.append(8000)

W0701 13:50:40.821790 140041750001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0701 13:50:40.887632 140041750001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0701 13:50:40.896921 140041750001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0701 13:50:40.928382 140041750001536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0701 13:50:40.940137 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 128)          128000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 600, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 383,194
Trainable params: 383,194
Non-trainable params: 0
_________________________________________________________________
None


Para crear la segunda definimos una estructura más compleja, formada por una consecución de tres pares de capas Dropout y LSTM. Además, en este caso se usan los pesos preentrenados de GloVe que hemos cargado anteriormente. 

In [16]:
# Hiperparámetros de la red E1
embed_dim = 50
lstm_out = 16

# Inicialización
model1 = Sequential()
# Capa de entrada
model1.add(Embedding(vocab, embed_dim, weights=[embedding_matrix], input_length=max_length, trainable=True))
# Capas Dropout y LSTM
model1.add(SpatialDropout1D(0.3))
model1.add(LSTM(lstm_out*3, dropout=0.1, return_sequences=True))
model1.add(SpatialDropout1D(0.2))
model1.add(LSTM(lstm_out*2, dropout=0.2, return_sequences=True))
model1.add(SpatialDropout1D(0.1))
model1.add(LSTM(lstm_out, dropout=0.1))
# Capa con dos salidas
#   Salida 0: emociones negativas
#   Salida 1: emociones positivas
model1.add(Dense(2,activation='softmax'))
# Compilamos el modelo
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# Mostramos su información
print(model1.summary())

# Guardamos el modelo y sus hiperparámetros en sus listas
models.append(model1)
batch_sizes.append(64)
n_registers.append(10000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 600, 50)           4206100   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 600, 50)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 600, 48)           19008     
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 600, 48)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 600, 32)           10368     
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 600, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 16)                3136      
__________

El tercer modelo tiene una estructura muy similar al primero, pero en este se usan también los pesos preentrenados de GloVe.

In [18]:
# Hiperparámetros de la red E2
embed_dim = 50
lstm_out = 196

# Inicialización
model2 = Sequential()
# Capa de entrada
model2.add(Embedding(vocab, embed_dim, weights=[embedding_matrix], input_length=max_length, trainable=True))
# Capa Dropout
model2.add(SpatialDropout1D(0.4))
# Capa LSTM
model2.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# Capa con dos salidas
#   Salida 0: emociones negativas
#   Salida 1: emociones positivas
model2.add(Dense(2,activation='softmax'))
# Compilamos el modelo
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# Mostramos su información
print(model2.summary())

# Guardamos el modelo en la lista de modelos, al igual que el tamaño de batch y el número de registros
models.append(model2)
batch_sizes.append(128)
n_registers.append(8000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 600, 50)           4206100   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 600, 50)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               193648    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 4,400,142
Trainable params: 4,400,142
Non-trainable params: 0
_________________________________________________________________
None


El último modelo cuenta con la misma estructura que el segundo, pero en este caso no se usan GloVe y los pesos se entrenarán directamente con el resto del modelo.

In [20]:
# Hiperparámetros
embed_dim = 50
lstm_out = 16

# Inicialización
model3 = Sequential()
# Capa de entrada
model3.add(Embedding(max_features, embed_dim,input_length = max_length))
# Capas Dropout y LSTM
model3.add(SpatialDropout1D(0.3))
model3.add(LSTM(lstm_out*3, dropout=0.1, return_sequences=True))
model3.add(SpatialDropout1D(0.2))
model3.add(LSTM(lstm_out*2, dropout=0.2, return_sequences=True))
model3.add(SpatialDropout1D(0.1))
model3.add(LSTM(lstm_out, dropout=0.1))
# Capas de salida
model3.add(Dense(2,activation='softmax'))
# Compilamos el modelo
model3.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# Mostramos su información
print(model3.summary())

# Guardamos el modelo y sus hiperparámetros en sus listas
models.append(model3)
batch_sizes.append(64)
n_registers.append(10000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 600, 50)           50000     
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 600, 50)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 600, 48)           19008     
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 600, 48)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 600, 32)           10368     
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 600, 32)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 16)                3136      
__________

Ahora pasamos a entrenar cada una de las redes. Para ello, se ha creado una función con el fin de automatizar el proceso, que comienza adaptando los datos de entrada y desordenando el conjunto de training para evitar que cada red se entrene con el mismo conjunto que las demás. Posteriormente hay que convertir cada palabra de las reseñas a conjuntos numéricos, y también obtener la puntuación de cada una.

Tras entrenar el modelo, se guarda en nuestra cuenta de Google Drive para su posterior análisis.

In [0]:
def create_models(tokenizer, n_registers, models, batch_sizes):
  
  # Para cada uno de los modelos
  for i in range(0, len(models)):
    
    # Desordenamos los registros del dataset de manera aleatoria
    train.sample(frac=1).reset_index(drop=True)
    
    # Nos quedamos con un número determinado de registros
    data_t = train.iloc[:n_registers[i]]
    print(data_t.count())
    
    # Convertimos cada palabra a secuencias de números
    X = tokenizer.texts_to_sequences(data_t['Review'].values)
    X = pad_sequences(X, maxlen=max_length)
    
    # Obtenemos las salidas esperadas 
    Y = pd.get_dummies(data_t['Reviewer_Score']).values
    
    # Entrenamos el modelo
    print("Training NN "+str(i))    
    models[i].fit(X, Y, epochs = 6, batch_size = batch_sizes[i], verbose = 2)
    
    # Guardamos el modelo en nuestro Google Drive
    models[i].save('/content/gdrive/My Drive/E'+str(i)+'.h5')
    print("Saved NN "+str(i))

Llamamos a la función con los modelos compilados anteriormente

In [26]:
create_models(tokenizer, n_registers, models, batch_sizes)

Review            8000
Reviewer_Score    8000
dtype: int64
Training NN 0
Epoch 1/6
 - 379s - loss: 0.4809 - acc: 0.7761
Epoch 2/6
 - 378s - loss: 0.4679 - acc: 0.7834
Epoch 3/6
 - 381s - loss: 0.4497 - acc: 0.7891
Epoch 4/6
 - 380s - loss: 0.4373 - acc: 0.8016
Epoch 5/6
 - 381s - loss: 0.4262 - acc: 0.8056
Epoch 6/6
 - 380s - loss: 0.4231 - acc: 0.8073
Saved NN 0
Review            10000
Reviewer_Score    10000
dtype: int64
Training NN 1
Epoch 1/6
 - 244s - loss: 0.5803 - acc: 0.7308
Epoch 2/6
 - 241s - loss: 0.5524 - acc: 0.7358
Epoch 3/6
 - 241s - loss: 0.5297 - acc: 0.7426
Epoch 4/6
 - 240s - loss: 0.5221 - acc: 0.7420
Epoch 5/6
 - 240s - loss: 0.5099 - acc: 0.7563
Epoch 6/6
 - 242s - loss: 0.5000 - acc: 0.7582
Saved NN 1
Review            8000
Reviewer_Score    8000
dtype: int64
Training NN 2
Epoch 1/6
 - 309s - loss: 0.5776 - acc: 0.7326
Epoch 2/6
 - 306s - loss: 0.5599 - acc: 0.7355
Epoch 3/6
 - 306s - loss: 0.5456 - acc: 0.7399
Epoch 4/6
 - 307s - loss: 0.5322 - acc: 0.7491
Epoch

Llegados a este punto, es hora de aplicar el conjunto de test sobre cada una de nuestras redes. De nuevo y por comodidad, se ha definido una función que nos permitirá testear también el ensemble sumando aquellos campos que no son iguales al comparar las predicciones con los resultados esperados para el conjunto de test.

In [0]:
def evaluate_error(model, tokenizer, test) -> np.float64:
  
  # Convertimos el texto del conjunto de test para que pueda ser evaluado por los modelos
  x_test = tokenizer.texts_to_sequences(test['Review'].values)
  x_test = pad_sequences(x_test, maxlen=max_length)
  y_test = test['Reviewer_Score'].values
  
  # Predecimos los resultados
  pred = model.predict(x_test, batch_size = 128)
  # Nos quedamos con la salida de la red que tiene mayor valor
  pred = np.argmax(pred, axis=1)
  # Calculamos el error y lo devolvemos, al igual que el conjunto de predicciones
  error = np.sum(np.not_equal(pred, y_test)) / y_test.size   
  return error, pred

Llegados a este punto, cargaremos los modelos que hemos entrenado, para obtener sus errores y predicciones asociadas al conjunto de test.

In [0]:
# Cargamos los modelos
models3 = []
models3.append(load_model('/content/gdrive/My Drive/E0.h5'))
models3.append(load_model('/content/gdrive/My Drive/E1.h5'))
models3.append(load_model('/content/gdrive/My Drive/E2.h5'))
models3.append(load_model('/content/gdrive/My Drive/E3.h5'))

predictions2 = []
errors2 = []
for x in range(0,4):
  # Obtenemos el error y las predicciones
  err, pred = evaluate_error(models3[x],tokenizer, test)
  predictions2.append(pred)
  errors2.append(err)

Los errores obtenidos para cada uno de los modelos son muy similares entre si. Esto se debe principalmente a que no han sido entrenados en profundidad, debido al enorme gasto de recursos que supone.

In [9]:
errors2

[0.24933333333333332,
 0.23266666666666666,
 0.24766666666666667,
 0.24066666666666667]

Teniendo cada una de las predicciones, podemos definir la lógica del ensemble y generar una predicción final. Por simplicidad definiremos el sistema de voto por la mayoría, y en caso de empate se decidirá el resultado de forma aleatoria.

In [0]:
from random import randint

size = len(predictions2[0])
pos = 0
n_models = len(predictions2)
fp = []
final_pred = []

for x in range(0,size):
  for i in range(0,n_models):
    if predictions2[i][x] == 1:
      # La variable pos cuenta el número de resultados etiquetados como positivos
      pos = pos + 1
  # Si el número de positivos es mayor que 2, el resultado final es 1
  if pos > 2:
    final_pred.append(1)
  # Si hay empate, se decide aleatoriamente
  elif pos == 2:
    final_pred.append(randint(0,1))
  # En otro caso, el resultado final es 0
  else:
    final_pred.append(0)
  fp.append(pos)
  pos = 0

Y ahora pasamos a calcular el error del ensemble sobre el conjunto de test. Vemos que la capacidad de acierto no mejora apenas.

In [15]:
y_test = test['Reviewer_Score'].values
error = np.sum(np.not_equal(final_pred, y_test)) / y_test.size
error

0.235

Podemos analizar más en detalle que está sucediendo si imprimimos por pantalla las predicciones de los modelos individuales y la predicción esperada. También se ha incluido el número de reseña.

In [0]:
equals = np.not_equal(final_pred, y_test)

Tal y como podemos apreciar, la gran mayoría de los errores provienen de predicciones en las que todos los modelos fallan. Esto es un indicador de que este tipo de ensembles simples no son del todo indicados para abordar problemas de esta clase con redes LSTM, a diferencia de los buenos resultados obtenidos en otros casos de naturaleza distinta al análisis de textos.

El principal problema radica en la complejidad en el entrenamiento. Como hemos comentado anteriormente, este tipo de redes necesitan de una gran cantidad de recursos, como por ejemplo memoria RAM, y tardan un tiempo significativamente elevado en conseguir calidad en los modelos a diferencia de otro tipo de problemas complejos como el reconocimiento de objetos en imágenes.

Sin embargo, introducir una arquitectura basada en ensemble siempre es beneficioso ya que aumenta la robustez del modelo, dado que la probabilidad de fallo es siempre menor usando un conjunto de modelos que con uno sólo. Algunas de las soluciones para conseguir mayor capacidad de acierto en el ensemble podrían ser:


*   Utilizar más recursos computacionales a la hora de entrenar el modelo.
*   Introducir nuevos modelos individuales.
*   Introducir otros tipos de modelos individuales, como máquinas de soporte vectorial (SVMs).



In [17]:
print("E0, E1, E2, E3, y")
for i in range(0,equals.size):
  if equals[i] == True:
    print(str(predictions2[0][i])+", "+str(predictions2[1][i])+", "+str(predictions2[2][i])+", "+str(predictions2[3][i])+", "+str(y_test[i])+"     "+str(i))

E0, E1, E2, E3, y
1, 1, 1, 1, 0     0
1, 1, 1, 1, 0     10
1, 1, 1, 1, 0     12
1, 1, 1, 1, 0     15
1, 0, 0, 1, 1     18
1, 1, 1, 1, 0     23
1, 1, 1, 1, 0     25
1, 0, 1, 0, 1     27
1, 1, 1, 1, 0     31
1, 0, 1, 0, 0     34
1, 1, 1, 1, 0     39
1, 1, 1, 1, 0     42
1, 1, 1, 1, 0     43
1, 0, 0, 1, 1     48
1, 1, 1, 1, 0     57
1, 0, 1, 1, 0     58
1, 0, 1, 0, 0     59
1, 1, 1, 1, 0     64
1, 1, 1, 1, 0     66
1, 1, 1, 1, 0     74
1, 1, 1, 1, 0     78
0, 0, 1, 1, 0     80
1, 1, 1, 1, 0     81
0, 0, 0, 0, 1     82
1, 1, 1, 1, 0     84
1, 0, 1, 1, 0     85
1, 1, 1, 1, 0     95
0, 0, 0, 0, 1     96
1, 1, 1, 1, 0     102
1, 1, 1, 1, 0     108
1, 0, 1, 1, 0     110
1, 0, 0, 1, 0     124
0, 0, 1, 0, 1     130
1, 1, 1, 1, 0     134
1, 1, 1, 1, 0     136
1, 1, 1, 1, 0     137
1, 1, 1, 1, 0     145
1, 0, 0, 1, 1     155
1, 0, 0, 1, 1     156
1, 1, 1, 0, 0     158
0, 0, 0, 0, 1     160
1, 0, 0, 0, 1     161
1, 1, 1, 1, 0     171
1, 0, 0, 0, 1     172
1, 1, 1, 1, 0     179
1, 0, 1, 1, 0     185